# Predikcia 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn.pipeline import Pipeline

In [2]:
pd.options.display.width = None
pd.options.display.max_columns = None

In [3]:
data_personal = pd.read_csv("data/personal_train.csv")
data_other = pd.read_csv("data/other_train.csv")

In [4]:
from Models import CombineDatasets as d1
from Models import DecodeJSONColumn as d2
from Models import ColumnDropper as d3
from Models import BinaryEncoder as d4
from Models import ClassFixer as d5
from Models import FixDates as d6
from Models import ComputeCurYear as d7
from Models import Replacer as d8
from Models import FillMeasuredColumns as d9
from Models import CountMeasuredHormones as d10
from Models import FillNanNumeric as d11
from Models import PredictNanNumeric as d12
from Models import Classifier as d13
from Models import QuantileReplacer as d14

In [5]:
bool_columns = [
    'FTI measured',
    'I131 treatment',
    'T3 measured',
    'goitre',
    'lithium',
    'on thyroxine',
    'pregnant',
    'tumor',
    'query on thyroxine',
    'thyroid surgery',
    'TSH measured',
    'query hyperthyroid',
    'TT4 measured',
    'on antithyroid medication',
    'sick',
    'T4U measured',
    'psych',
    'query hypothyroid'
]

In [6]:
ppl = Pipeline([
    ('combination', d1.CombineDatasets()),
#     ('json_decode', d2.DecodeJSONColumn('medical_info')),
#     ('drop_columns', d3.ColumnDropper(['hypopituitary', 'TBG measured', 'TBG'])),
#     ('binary_fixer', d4.BinaryEncoder(bool_columns)),
#     ('class_fix', d5.ClassFixer()),
#     ('fix_dates', d6.FixDates()),
#     ('compute_cur_year', d7.ComputeCurYear()),
#     ('fti_question_mark_to_nan', d8.Replacer(['FTI'], '?', float('nan'), float)),
#     ('fti_question_mark_to_nan1', d8.Replacer(['FTI'], np.nan, float('nan'), float)),
#     ('fill_measured_columns', d9.FillMeasuredColumns()),
#     ('count_measured', d10.CountMeasuredHormones('measured_hormones')),
    
#     # Fill hormony
#     ('fill_tt4', d11.FillNanNumeric('mean', 'TT4')),
#     ('fill_tsh', d11.FillNanNumeric('median', 'TSH')),
#     ('predicted_t3', d12.PredictNanNumeric('T3', ['TT4', 'TSH'])),
#     ('predicted_t4u', d12.PredictNanNumeric('T4U', ['TT4', 'TSH'])),
#     ('predicted_fti', d12.PredictNanNumeric('FTI', ['TT4', 'TSH', 'T4U'])),
    
#     # Fill pregnant
#     ('pregnant_predict', d13.Classifier('pregnant', ['T3', 'T4U', 'TT4'])),
    
#     # Fill other variables
#     ('fill_tumor', d11.FillNanNumeric('median', 'tumor')),
#     ('fill_lithium', d11.FillNanNumeric('mean', 'lithium')),
#     ('fill_on_thyroxine', d11.FillNanNumeric('mean', 'on thyroxine')),
#     ('fill_goitre', d11.FillNanNumeric('most_frequent', 'goitre')),
#     ('fill_q_on_thyroxine', d11.FillNanNumeric('mean', 'query on thyroxine')),
    
#     #Replace outlier
#     ('replace_t3', d14.QuantileReplacer('T3')),
#     ('replace_age', d14.QuantileReplacer('age')),
#     ('replace_t4u', d14.QuantileReplacer('T4U')),
#     ('replace_tsh', d14.QuantileReplacer('TSH')),
#     ('replace_tt4', d14.QuantileReplacer('TT4')),
#     ('replace_fti', d14.QuantileReplacer('FTI')),
    
])

In [7]:
#data = ppl.fit(data_personal, data_other)

In [8]:
data.fit_transform(data_personal, data_other)

,Unnamed: 0_x,name,address,age,sex,date_of_birth,FTI measured,I131 treatment,T3,T3 measured,T4U,TBG,TBG measured,TSH,TT4,Unnamed: 0_y,capital-gain,capital-loss,class,education,education-num,fnlwgt,goitre,hours-per-week,hypopituitary,lithium,marital-status,medical_info,native-country,occupation,on thyroxine,pregnant,query on thyroxine,race,referral source,relationship,thyroid surgery,tumor,workclass
0,0,Terry Terry,"11818 Lori Crossing Apt. 802\nPughstad, DC 78165",24.0,M,94-05-02,f,f,1.6,t,NaN,?,f,0.100,182.0,1476.0,0.0,0.0,negative.|2815,HS-grad,9.0,144844.0,f,40.0,f,f,Married-civ-spouse,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Craft-repair,f,f,f,White,SVHC,Husband,f,f,Private
1,1,Stephen Lalk,"PSC 4657, Box 5446\nAPO AP 58412",44.0,M,1974-04-29 00 00 00,t,f,1.3,t,0.94,?,f,2.000,136.0,444.0,0.0,0.0,negative.|3345,Bachelors,13.0,335067.0,f,50.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Sales,f,f,f,White,SVHD,Not-in-family,f,f,Private
2,2,Abraham Bruce,Unit 9759 Box 9470\nDPO AP 45549,56.0,M,1962-06-09,f,f,NaN,f,NaN,?,f,NaN,NaN,1658.0,0.0,0.0,negative.|262,Prof-school,15.0,186035.0,f,30.0,f,f,Married-civ-spouse,"{'TSH measured':'f','query hyperthyroid':'f','...",United-States,Sales,f,f,f,White,other,Husband,f,f,Self-emp-not-inc
3,3,Edith Boudreaux,"137 Lewis Flat Suite 762\nWest Elizabeth, AL 3...",71.0,F,1946-12-16 00 00 00,t,f,1.8,t,1.16,?,f,1.600,136.0,2203.0,0.0,0.0,negative.|2228,Bachelors,13.0,36601.0,f,50.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Prof-specialty,f,f,f,White,SVI,Not-in-family,f,f,Private
4,4,Janet Washington,"995 Frank Stravenue\nSouth Matthewport, TX 81402",63.0,F,1955-10-15,t,f,1.8,t,0.85,?,f,0.670,107.0,2275.0,0.0,0.0,negative.|3085,HS-grad,9.0,29810.0,f,50.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Other-service,f,f,f,White,other,Not-in-family,f,f,Private
5,5,Margaret Chabot,"1781 Meredith Skyway Suite 328\nCordovaburgh, ...",63.0,F,1955-05-22,f,t,1.6,t,NaN,?,f,2.900,101.0,1775.0,0.0,0.0,negative.|873,Some-college,10.0,518530.0,f,40.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Adm-clerical,t,f,f,White,Other,Own-child,f,f,Private
6,6,James Wilson,"2415 Elizabeth Knoll Suite 030\nCordovafort, C...",70.0,M,1948-05-03,t,f,1.0,t,0.88,?,f,3.600,60.0,859.0,0.0,0.0,negative.|3293,Doctorate,16.0,343721.0,f,30.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",?,Prof-specialty,f,f,f,White,SVI,Not-in-family,f,f,Self-emp-not-inc
7,7,Roy Wilson,"49824 Kim View\nWest Ericborough, VT 42457",54.0,?,1964-06-07,t,f,1.9,t,0.99,?,f,2.700,111.0,1261.0,4650.0,0.0,negative.|1696,12th,8.0,192704.0,f,50.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,Exec-managerial,f,f,f,White,SVI,Not-in-family,f,f,Private
8,8,Colleen Satterwhite,"53231 Matthew Spur Apt. 079\nNorth Brian, MA 3...",48.0,F,1970-06-07 00:00:00,t,f,2.7,t,1.20,?,f,1.800,124.0,1913.0,0.0,0.0,negative.|3250,Some-college,10.0,153167.0,f,40.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",United-States,?,f,f,f,Black,SVI,Own-child,f,f,?
9,9,Judy Smith,USNV Gallegos\nFPO AA 99743,55.0,F,1963-06-02,t,f,2.3,t,0.97,?,f,1.800,98.0,2071.0,0.0,0.0,negative.|774,12th,8.0,176321.0,f,40.0,f,f,Never-married,"{'TSH measured':'t','query hyperthyroid':'f','...",Mexico,Adm-clerical,f,f,f,White,SVI,Own-child,f,f,Private
